# Klasyfikator wieloklasowy - Image Recognition

Celem tego notebook'a jest budowa modelu klasyfikacji obrazów przedstawiających różne elementy ubioru. W tym celu wykorzystamy kolejny popularny zbiór danych w dziedzinie sieci neuronowych: Fashion MNIST

Fashion-MNIST to zestaw danych z obrazkami artykułów Zalando - składający się z zestawu treningowego 60 000 przykładów zestawu testowego 10 000 przykładów. Każdy przykład to obraz w skali szarości 28 x 28, powiązany z jedną etykietą z 10 klas.
<br><br>
<div style="width: 100%; text-align: center">

| Label | Description |
|-------|-------------|
| 0     | T-shirt/top |
| 1     | Trouser     |
| 2     | Pullover    |
| 3     | Dress       |
| 4     | Coat        |
| 5     | Sandal      |
| 6     | Shirt       |
| 7     | Sneaker     |
| 8     | Bag         |
| 9     | Ankle boot  |

</div>

# 1. Import bibliotek

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
sns.set()
tf.__version__

# 2. Załadowanie danych i wstępna eksploracja

In [ ]:
(X_train, y_train), (X_test, y_test) = load_data()

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')
print(f'X_train[0] shape: {X_train[0].shape}')

In [ ]:
X_train[0]

In [ ]:
y_train[:10]

In [ ]:
plt.imshow(X_train[100], cmap='gray_r')
plt.axis('off')

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

plt.figure(figsize=(18, 13))
for i in range(1, 11):
    plt.subplot(1, 10, i)
    plt.axis('off')
    plt.imshow(X_train[i-1], cmap='gray_r')
    plt.title(class_names[y_train[i-1]], color='black', fontsize=16)
plt.show()

In [ ]:
X_train = X_train / 255.
X_test = X_test / 255.

# 3. Budowa modelu

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

**epochs** - ile razy zestaw treningowy zostanie przetworzony przez model. Przy każdej iteracji optymalizator próbuje dopasować wagi, aby funkcja celu została zminimalizowana.

**batch_size** - liczba przykładów treningowych po której następuje aktualizacji wag

**validation_split** - procent danych użytych do walidacji

**vaidation_data** - dane, na których zostanie przeprowadzona walidacja

**callbacks** - wykorzystane wywołania zwrotne, np.
- **EarlyStopping** - zatrzymanie nauki, przy zaprzestaniu poprawy obserwowanej metryki
- **ModelCheckpoint** - zapisywanie najlepszych wag
- **TensorBoard** - interfejs do dokładnej eksploracji okresu nauki

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=2)
mc = ModelCheckpoint(filepath="best_weights.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

history = model.fit(X_train, y_train, epochs=15, batch_size=10, validation_split=0.2, callbacks=[mc])

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
acc

In [ ]:
model.load_weights("best_weights.hdf5")
loss, acc = model.evaluate(X_test, y_test, verbose=0)
acc

In [ ]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

# 4. Ocena modelu

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy')
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(test_acc)

# 5. Predykcja na podstawie modelu:

**model.predict()** - pozwala zwrócić prawdopodobieństwo danej klasy

In [ ]:
predictions = model.predict(X_test)
predictions

In [ ]:
predictions_df = pd.DataFrame(predictions)
predictions_df.head()

In [ ]:
classes = np.argmax(predictions, axis=1)
classes

# 6. Eksploracja predykcji

In [ ]:
idx = 999

if classes[idx] == y_test[idx]:
    color = 'green'
else:
    color = 'red'

fig = go.Figure()
fig.add_trace(go.Bar(x=class_names, y=predictions_df.iloc[idx], orientation='v',
                     marker_color=color))
fig.update_layout(width=800, title=f'Predykcja: {class_names[classes[idx]]}')
fig.show()

plt.imshow(X_test[idx], cmap='gray_r')
plt.axis('off')

# 7. Błędnie sklasyfikowane obrazy

In [ ]:
misclassified = []
for idx, _ in enumerate(X_test):
    if classes[idx] != y_test[idx]:
        misclassified.append(idx)

index_mapper = {}


for idx, idx_real in enumerate(misclassified):
    index_mapper[idx] = idx_real

idx = 1

fig = go.Figure()
fig.add_trace(go.Bar(x=class_names,
                     y=predictions_df.iloc[index_mapper[idx]],
                     orientation='v',
                     marker_color='red'))

fig.update_layout(width=800,
                  title=(f' Etykieta: {class_names[y_test[index_mapper[idx]]]} ~ Predykcja: {class_names[classes[index_mapper[idx]]]}'))
fig.show()

plt.imshow(X_test[index_mapper[idx]], cmap='gray_r')
plt.axis('off')

# 8. Klika możliwości poprawy modeli:
- zwiększenie liczby epok
- zwiększenie/zmniejszenie parametrów uczenia, stronjenie hiperparametrów
- zwiększenie/zmniejszenie liczby neuronów wewnątrz wartsw uktytych
- zwiększenie/zmniejszenie parametru **batch_size**, czyli rozmiaru wsadu
- zwiększenie/zmniejszenie liczby warstw ukrytych
- zastosowanie regularyzacji (L1 - lasso, L2 - ridge), warstwy Dropout